## AI Chatbot Activity Guide for Students

### Activity 1: Basic Chatbot Interaction

**Aim:**
Learn how to interact with an AI-powered chatbot by entering simple questions or greetings.

**Shortfall:**
You might notice that the chatbot gives unexpected or unrelated responses when given very general or open-ended questions.

**Resolution in Next Activity:**
To address this, you'll learn about conversational context in the next step, helping the chatbot generate better responses.

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

# Load pre-trained DialoGPT model
tokenizer = AutoTokenizer.from_pretrained("microsoft/DialoGPT-medium")
model = AutoModelForCausalLM.from_pretrained("microsoft/DialoGPT-medium")

print("🤖 Chatbot is ready! Type 'quit' to stop.")

# Initialize chat history
chat_history_ids = None

while True:
    # Get user input
    user_input = input("You: ")
    if user_input.lower() in ["quit", "exit"]:
        print("👋 Chatbot session ended.")
        break

    # Encode user input and append to chat history
    new_input_ids = tokenizer.encode(user_input + tokenizer.eos_token, return_tensors='pt')

    # Append tokens to the chat history
    bot_input_ids = torch.cat([chat_history_ids, new_input_ids], dim=-1) if chat_history_ids is not None else new_input_ids

    # Generate response
    chat_history_ids = model.generate(
        bot_input_ids,
        max_length=1000,
        pad_token_id=tokenizer.eos_token_id,
        do_sample=True,
        top_k=100,
        temperature=0.75
    )

    # Decode and print response
    response = tokenizer.decode(chat_history_ids[:, bot_input_ids.shape[-1]:][0], skip_special_tokens=True)
    print(f"Chatbot: {response}")


### Activity 2: Improving Attention Masks

**Aim:**
Resolve the attention mask problem by explicitly defining masks, making the responses more coherent.

**Shortfall:**
DialoGPT still gives conversational but nonsensical responses when asked factual or simple questions.

**Resolution in Next Activity:**
In the following activity, you'll switch to GPT-Neo, a model more suited to general knowledge questions and clearer responses.

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

# Load DialoGPT-medium model and tokenizer
tokenizer = AutoTokenizer.from_pretrained("microsoft/DialoGPT-medium")
model = AutoModelForCausalLM.from_pretrained("microsoft/DialoGPT-medium")

# Ensure pad_token_id is defined properly
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

print("🤖 Chatbot ready! Type 'quit' to exit.")

# Start chat
chat_history_ids = None

while True:
    user_input = input("You: ")
    if user_input.lower() in ["quit", "exit"]:
        print("👋 Chatbot session ended.")
        break

    # Encode user input and manage attention mask
    new_user_input_ids = tokenizer.encode(user_input + tokenizer.eos_token, return_tensors='pt')

    # Append user input to chat history
    bot_input_ids = torch.cat([chat_history_ids, new_user_input_ids], dim=-1) if chat_history_ids is not None else new_user_input_ids

    # Create attention mask explicitly
    attention_mask = torch.ones(bot_input_ids.shape, dtype=torch.long)

    # Generate a response from DialoGPT
    chat_history_ids = model.generate(
        bot_input_ids,
        attention_mask=attention_mask,
        max_length=1000,
        pad_token_id=tokenizer.eos_token_id,
        do_sample=True,
        top_p=0.9,
        top_k=50,
        temperature=0.7
    )

    # Decode and display the response
    response = tokenizer.decode(chat_history_ids[:, bot_input_ids.shape[-1]:][0], skip_special_tokens=True)
    print(f"Chatbot: {response}")


### Activity 3: Chatbot using GPT-Neo

**Aim:**
Build an interactive chatbot with GPT-Neo for better responses to general-knowledge and factual questions.

**Shortfall:**
GPT-Neo responses may be slow and sometimes irrelevant due to its nature as a general text-generation model, not optimized specifically for conversational interaction.

**Resolution in Next Activity:**
The next activity introduces a professional-grade conversational model using Google's Gemini API, enhancing response quality significantly.

In [ ]:
from transformers import pipeline

# Load GPT-Neo
generator = pipeline('text-generation', model='EleutherAI/gpt-neo-1.3B')

print("🤖 AI is ready! Type 'quit' to exit.")

while True:
    question = input("\nYou: ")
    if question.lower() in ['quit', 'exit']:
        print("👋 Ending session.")
        break

    prompt = f"Q: {question}\nA:"
    response = generator(
        prompt,
        max_length=100,
        temperature=0.7,
        do_sample=True,
        truncation=True  # explicitly truncates long inputs
    )

    answer = response[0]['generated_text'].split('A:')[1].split('Q:')[0].strip()
    print(f"AI: {answer}")


### Activity 4: Gemini API Chatbot (Free Version)

**Aim:**
Set up a chatbot using Google's Gemini API, significantly improving response coherence, speed, and reliability.

**Shortfall:**
Initial implementation might not handle conversational context optimally, potentially losing the conversation history or context.

**Resolution in Next Activity:**
The final activity implements an improved conversation management system provided by Gemini to ensure coherent multi-turn conversations.

In [ ]:
import google.generativeai as genai

# Replace with your actual API key
genai.configure(api_key="Your_API_Key")


for m in genai.list_models():
    if 'generateContent' in m.supported_generation_methods:
        print(m.name)

model = genai.GenerativeModel('gemini-2.0-flash-lite')

def chat_with_gemini():
    conversation_history = []
    while True:
        user_input = input("You: ")
        if user_input.lower() == "exit":
            break

        conversation_history.append({"role": "user", "parts": user_input})

        try:
            response = model.generate_content(conversation_history)
            gemini_response = response.text
            print("Gemini:", gemini_response)
            conversation_history.append({"role": "model", "parts": gemini_response})

        except Exception as e:
            print(f"Error: {e}")

chat_with_gemini()

### Activity 5: Optimized Gemini Conversational Chatbot

**Aim:**
Enhance the Gemini chatbot using Gemini's built-in conversational management (`start_chat`) for smooth, multi-turn interactions.

**Benefits:**
- Seamless management of conversational context
- Highly coherent and accurate responses
- User-friendly and simplified code structure

In [ ]:
import google.generativeai as genai

# Configure your Gemini API key
genai.configure(api_key="YOUR_API_KEY")

# Initialize Gemini model
model = genai.GenerativeModel('gemini-2.0-flash-lite')

def chat_with_gemini():
    conversation = model.start_chat(history=[])

    print("🤖 Gemini AI Chatbot is ready! (type 'exit' to quit)\n")

    while True:
        user_input = input("You: ")
        if user_input.lower() == "exit":
            print("👋 Session ended.")
            break

        try:
            response = conversation.send_message(user_input)
            print(f"Gemini: {response.text}")

        except Exception as e:
            print(f"Error: {e}")

chat_with_gemini()


### Reflection

Through these activities, you've explored common chatbot issues and progressively resolved them by adopting advanced models and proper conversational context management. You've also explored selecting appropriate AI models based on your conversational needs.

